# Concurrent Database Connections
DocTable makes it easy to establish concurrent database connections from different processes. DocTable objects can be copied as-is from one process to another, except that you must call `.reopen_engine()` to initialize in process thread. This removes now stale database connections (which are _not_ meant to traverse processes) from the engine connection pool.

You may also want to use a large timeout using the timeout argument of the doctable constructor (provided in seconds).

In [1]:
import sqlalchemy
from multiprocessing import Process
import os
import random
import string
import dataclasses
import time
import sys
sys.path.append('..')
import doctable

In [2]:
import datetime
@dataclasses.dataclass
class SimpleRow(doctable.DocTableRow):
    id: int = doctable.IDCol()
    updated: datetime.datetime = doctable.AddedCol()
    process: str = doctable.Col()
    number: int = doctable.Col()

tmp = doctable.TempFolder('exdb')
db = doctable.DocTable(schema=SimpleRow, target=tmp.joinpath('tmp_concurrent.db'), new_db=True, timeout=60)

In [3]:
def thread_func(numbers, db):
    process_id = ''.join(random.choices(string.ascii_uppercase, k=2))
    print(f'starting process {process_id}')
    db.reopen_engine() # create all new connections
    for num in numbers:
        db.insert({'process': process_id, 'number': num})
        time.sleep(0.01)

numbers = list(range(1000)) # these numbers are to be inserted into the database
        
db.delete()
with doctable.Distribute(5) as d:
    d.map_chunk(thread_func, numbers, db)
db.head(20)

starting process WF
starting process WY
starting process HA
starting process ZV
starting process GK


,id,updated,process,number
0,1,2021-06-02 14:04:02.889887,WF,400
1,2,2021-06-02 14:04:02.895034,WY,600
2,3,2021-06-02 14:04:02.898325,HA,800
3,4,2021-06-02 14:04:02.903394,WF,401
4,5,2021-06-02 14:04:02.908172,WY,601
5,6,2021-06-02 14:04:02.911528,HA,801
6,7,2021-06-02 14:04:02.911932,ZV,0
7,8,2021-06-02 14:04:02.916282,WF,402
8,9,2021-06-02 14:04:02.921119,WY,602
9,10,2021-06-02 14:04:02.923891,GK,200
